In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [4]:
#Variable 
var="tg_mean"

files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

In [5]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_tg_mean_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Get the percentiles

In [ ]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [ ]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [ ]:
#dfmean45

In [ ]:
dfmean45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

In [7]:
dfmean45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

# FOR RCP85 

In [ ]:
dsEns85= ens.create_ensemble(files85)
#dsEns85

## Get the percentiles

In [ ]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [ ]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [ ]:
#dfmean85

In [ ]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [8]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# OBSERVED DATA

In [ ]:
dsOBs= xr.open_mfdataset(filesobs)

In [24]:
dsOBs

NameError: name 'dsOBs' is not defined

## Convert in Dataframe

In [ ]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [ ]:
dfObs = dfmeanObs.rename(columns={'tg_mean': 'tg_mean_Obs'})

In [25]:
dfObs

,lat,lon,time,tg_mean_Obs
0,83.125000,-74.541672,1950-01-01,251.429901
1,83.125000,-74.541672,1951-01-01,252.994797
2,83.125000,-74.541672,1952-01-01,253.964935
3,83.125000,-74.541672,1953-01-01,253.616013
4,83.125000,-74.541672,1954-01-01,253.628616
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,282.584686
16464828,41.708332,-82.208336,2010-01-01,283.870300
16464829,41.708332,-82.208336,2011-01-01,284.144257
16464830,41.708332,-82.208336,2012-01-01,285.570129


In [ ]:
dfObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [20]:
dfObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge 2 rcp dataframes

In [10]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [11]:
df2m

,lat,lon,time,tg_mean_p10_45,tg_mean_p50_45,tg_mean_p90_45,tg_mean_p10_85,tg_mean_p50_85,tg_mean_p90_85
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655,257.056732,258.643677,260.095032
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644,256.838013,258.383789,259.087128
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579,257.158539,258.634918,259.196228
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734,257.833679,258.571075,259.167938
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162,258.513580,259.165405,260.001831
...,...,...,...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594,289.641998,290.565857,292.025970
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133,289.080017,290.811249,291.779266
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590,288.290466,290.388062,291.735809
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461,289.418976,290.153625,291.091431


# Cut by region for rcp45, 85 and Obs

## Get the lat lon regions

In [12]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [13]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [14]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

## Create DF by region

In [15]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [16]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [17]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

## Cutting rcp45 and 85 by region and getting df with 4th degree polynomial

In [18]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [28]:
dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 


In [30]:
dfmeanObs

index        lat        lon  \
time       NOM                                                                  
1950-01-01 Golfe du Saint-Laurent         436015.147368  49.510526 -62.620617   
           Pessière à lichens             375910.525566  54.191234 -72.343737   
           Pessière à mousses             417290.567439  50.964888 -69.743773   
           Sapinière à bouleau blanc      447153.580057  48.632600 -72.042851   
           Sapinière à bouleau jaune      458478.780609  47.748361 -72.643701   
...                                                 ...        ...        ...   
2013-01-01 Toundra arctique herbacée      281055.840659  61.591024 -73.922896   
           Toundra forestière             344270.715669  56.661788 -70.441977   
           Érablière à bouleau jaune      475718.926630  46.401570 -74.337638   
           Érablière à caryer cordiforme  487501.517094  45.482550 -73.968663   
           Érablière à tilleul            480278.582692  46.047276 -72.752727   

                                          index_right     OBJECTID  \
time       NOM                                                       
1950-01-01 Golfe du Saint-Laurent           67.000000    68.000000   
           Pessière à lichens             1267.831852  1268.831852   
           Pessière à mousses             1203.813659  1204.813659   
           Sapinière à bouleau blanc       626.488853   627.488853   
           Sapinière à bouleau jaune        65.891967    66.891967   
...                                               ...          ...   
2013-01-01 Toundra arctique herbacée      1290.992308  1291.992308   
           Toundra forestière             1288.972051  1289.972051   
           Érablière à bouleau jaune         9.736413    10.736413   
           Érablière à caryer cordiforme     5.149573     6.149573   
           Érablière à tilleul               7.817308     8.817308   

                                            SUPERFICIE  Shape_Leng  \
time       NOM                                                       
1950-01-01 Golfe du Saint-Laurent         1.303756e+07   85.669342   
           Pessière à lichens             2.842746e+07   58.912861   
           Pessière à mousses             3.872688e+07   93.425670   
           Sapinière à bouleau blanc      9.823173e+06   70.803812   
           Sapinière à bouleau jaune      1.088530e+07   85.663450   
...                                                ...         ...   
2013-01-01 Toundra arctique herbacée      3.693841e+06   28.445817   
           Toundra forestière             2.015250e+07   53.013537   
           Érablière à bouleau jaune      3.991023e+06   25.314764   
           Érablière à caryer cordiforme  1.068052e+06    8.240411   
           Érablière à tilleul            2.162467e+06   18.706636   

                                          Shape_Area  tg_mean_Obs  
time       NOM                                                     
1950-01-01 Golfe du Saint-Laurent          16.230097   275.370544  
           Pessière à lichens              39.729178   267.888397  
           Pessière à mousses              50.125340   271.220642  
           Sapinière à bouleau blanc       12.073276   273.587067  
           Sapinière à bouleau jaune       13.153702   275.088043  
...                                              ...          ...  
2013-01-01 Toundra arctique herbacée        6.200459   264.634003  
           Toundra forestière              29.877241   268.702301  
           Érablière à bouleau jaune        4.681339   277.218567  
           Érablière à caryer cordiforme    1.225917   279.807587  
           Érablière à tilleul              2.515389   278.659668  

[704 rows x 9 columns]

In [ ]:
listdf2m = []

for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, df2m, on=["lat","lon"])
    dfdataregObs = pd.merge(dfregion, dfObs, on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region")
    dfmeanUA45 = dfdatareg45
    dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    dfmeanObs = dfdataregObs.groupby(["time", name]).mean() 
    dfmeanObs2 = dfmeanObs.reset_index()
    print("Merging 3 cases")
    df3m = pd.merge(dfmean45UA2, dfmeanObs2, how='left', on= ["time", name] )
    print ("4/8... Split by subregion")
    df3ma = df3m[['time', name, 'tg_mean_p10_45', 'tg_mean_p50_45', 'tg_mean_p90_45',
                               'tg_mean_p10_85', 'tg_mean_p50_85', 'tg_mean_p90_85', 'tg_mean_Obs']]
#    for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
#        df3ma[var+p] = df3ma.groupby(name)[var+p].transform(lambda sr: polysmooth(years,sr,4) - 273.15)
#    df3ma.to_csv(replace_all(name2,d)+var+".csv")
#    continue
    df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15
    df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)
    for name2,dfsr in df3ma.groupby(name):
        print ("                   "+name2, name)
        dfsrc = dfsr.copy()
        print ("6/8... 4th Polynomial - rcp45,85")
        years = dfsrc.time.dt.year
        dfsrc["time"] = years#.values
        for p in ('_p10_45', '_p50_45', '_p90_45', '_p10_85', '_p50_85', '_p90_85'):
            dfsrc[var+p] = polysmooth(years,dfsrc[var+p],4) - 273.15
            dfsrc[var+p] = round(dfsrc[var+p], 2) 
        print(" -----------------------8/8 Creating CSV file for sub-region " +name2)
        dfsrc.to_csv(replace_all(name2,d)+"_"+var+"_annual.csv")
    
    
                            
       

------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
3/8... Get mean by year for every sub-region


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3ma[var+"_Obs"] = df3ma[var+"_Obs"]- 273.15

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3ma[var+"_Obs"] = round(df3ma[var+"_Obs"], 2)



Merging 3 cases
4/8... Split by subregion
                   ABITIBI-TEMISCAMINGUE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ABITIBI-TEMISCAMINGUE
                   BAS-SAINT-LAURENT NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region BAS-SAINT-LAURENT
                   CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
                   COTE-NORD NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region COTE-NORD
                   ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL NM_REG_FOR
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
                   GASPESIE-ILES-DE-LA-MADELEINE NM_REG_FOR
6/8... 4th Polynomial - rcp45,85

                   08666 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08666
                   08751 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08751
                   08762 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08762
                   08763 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08763
                   08764 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 08764
                   09351 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 09351
                   09352 PER_NO_UA
6/8... 4th Polynomial - rcp45,85
 -----------------------8/8 Creating CSV file for sub-region 09352
                   09471 PER_NO_UA
6/8... 4th Polynomia